In [2]:
import logic.pulsed.pulse_objects as po
from logic.pulsed.sampling_functions import SamplingFunctions as SF
import time
import matplotlib.pyplot as plt
pml = pulsedmeasurementlogic
pmal = pulsedmasterlogic

In [3]:
Sync_time = (16e-9+476.5/1.25e9)
print("Sync",Sync_time)

Rearm_time = (40/1.25e9)
print("Rearm_time",Rearm_time)


Sync 3.972e-07
Rearm_time 3.2e-08


#### Current  philosophy of pulse sequence creation
1. For all the pulsed measurement that involve possible multiple phases and different pulse lengths - it is preferred and also implement right now - a long AWG sequence covering all taus is run with a single trigger from the PulseStreamer

2. For T1 sequence since it is very long and we have issues possibly due to memory - we individually trigger the AWG pi pulse before every needed pulse with the AWG sync time added in - this is also how the PODMR logic is implemented (albeit for many different freqs. so actually a different ensemble is being triggered unlike T1) - uses the sample_load_multi_short_sine_AWG function.

3. This is also valid for any other pulse sequence imaginable where individual triggering would be preferred - as long as the length of the triggerable ensembles are equal or only single ensemble is necessary.

4. Both cam be run in AWG switch mode = True (Zaswa is constantly on and AWG control the pulsing - signficantly better pulse shape and results compared to the other mode - highly recommended!).

5. Tracking mode for qAFM is also implemented such that the MW switch is always on and the AWG is controlling the on - off times

#### Define pulse block creation class

In [29]:
class qudi_pulse_block:

    def __init__(self, pi_pulse=1e-9, pi_half_pulse=1e-9, three_pi_half_pulse=1e-9, AWG_switch_mode=True, awg_sync_time=16e-9 + 476.5/1.25e9, laser_waiting_time=1.5e-6, mw_waiting_time=1e-6, read_out_time=3e-6,
                LO_freq=3e9, target_freq=2.88e9, target_freq_DQ=None, power=-20):
        
        self.pi_pulse = pi_pulse
        self.pi_half_pulse = pi_half_pulse
        self.three_pi_half_pulse = three_pi_half_pulse
        self.AWG_switch_mode = AWG_switch_mode # to use the AWG as the switching device - will leave the Zaswa switch always on
        self.awg_sync_time = awg_sync_time #Has to be determined with sample clock
        self.laser_waiting_time = laser_waiting_time
        self.mw_waiting_time = mw_waiting_time
        self.read_out_time = read_out_time
        self.laser_volt = podmrlogic.laser_power_voltage
        self.LO_freq = LO_freq
        self.target_freq = target_freq
        self.target_freq_DQ = target_freq_DQ #Frequency of the +1 peak for double quantum sequences
        self.power = power
        print("Using laser voltage: ", self.laser_volt,"V")
        print("Using LO_freq: ", self.LO_freq/1e9,"GHz")
        print("Using target_freq: ", self.target_freq/1e9,"GHz")
        print("Using power: ", self.power,"dBm")
        
        self.channel_names_PS = {'Laser': 'd_ch1',
                                 'TT_Start': 'd_ch2',
                                 'TT_Next': 'd_ch3',
                                 'ASC500': 'd_ch4',
                                 'MW_Switch': 'd_ch5',
                                 'MW_Trig': 'd_ch6',                                
                                 'AWG_Trig': 'd_ch7',
                                 'AWG_Clock': 'd_ch8'
                                }
        
    def ElementPS(self, channels={}, length=1e-9, laser_power=None, phase=0, DQ=False):
        """PulseBlock element list maker for PulseStreamer upload. Also makes the phase duration list for the AWG. Also has
            a marker to define, if the DQ frequency has to be played.
        """
        a_ch = {'a_ch1': SF.DC(laser_power) if laser_power else SF.DC(self.laser_volt), 'a_ch2': SF.DC(0)}
        d_ch = { 'd_ch1': False, 'd_ch2': False, 'd_ch3': False, 'd_ch4': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
        user_MW_true = False
        if self.AWG_switch_mode:
            if 'MW_Switch' in channels.keys():
                user_MW_true = channels['MW_Switch']
            channels.update({'MW_Switch': True})
        for key in channels:
            PSkey = self.channel_names_PS[key]
            if 'a_' in key:
                a_ch[PSkey] = channels[key]
            else:
                d_ch[PSkey] = channels[key]
        
        self.BlockPS.append(po.PulseBlockElement(init_length_s=length, pulse_function=a_ch, digital_high=d_ch))
        if isinstance(phase, tuple):
            self.PhaseDuration.append((phase[0], length/2,DQ, user_MW_true))
            self.PhaseDuration.append((phase[1], length/2,DQ, user_MW_true))
        else:
            self.PhaseDuration.append((phase, length, DQ, user_MW_true))
        
    def AWG_reset(self):
        """ Resets all pulsing
        """
        afm_scanner_logic._AWG.pulser_off()
        afm_scanner_logic._AWG.instance.init_all_channels()
        afm_scanner_logic._mw.off()
        
    def AWG_start(self):
        """ Starts AWG in triggered mode, sets and starts LO
        """
        afm_scanner_logic._pulsed_master_AWG.pulsedmeasurementlogic().pulsegenerator().pulser_on(trigger=True)
        afm_scanner_logic._mw.set_cw(frequency=self.LO_freq, power=self.power)
        afm_scanner_logic._mw.cw_on()
        
    def sample_load_large_sine_AWG(self, name):
        """Function to loop through the PhaseDuration list defined with ElementPS for each measurement.
            A list of all these small steps are made into an ensemble by load_large_sine_seq and is ready to be 
            played by trigger.
            One big ensemble covering the entire tau sweep that is triggered once before every sweep. Not before every tau instance.
        """
        #Create large pulse block for the AWG
        self.AWG_reset()
        large_seq = []
        #skipping first element since it is the AWG sync time
        #skipping last element since AWG has a rearm time before the next averaging repetition trigger arrives
        for PhaseElement in self.PhaseDuration[1:-1]:
            phase, duration, DQ, user_MW_true = PhaseElement
            delta = abs(self.LO_freq - self.target_freq)
            if DQ:
                init_phase = (100.00,0.00) # allows to drive the other transition since LO is defined to be in center in DQ methods
            else:
                init_phase = (0.00,100.00)
            if self.AWG_switch_mode:
                seq_part = {'channel_info' : [   {'name': 'a_ch0', 'amp': 2.0 if user_MW_true else 0.0, 'freq': delta, 'phase': init_phase[0]+phase},
                             {'name': 'a_ch1', 'amp': 2.0 if user_MW_true else 0.0, 'freq': delta, 'phase': init_phase[1]+phase}],
                             'duration' : duration}
            else:
                seq_part = {'channel_info' : [   {'name': 'a_ch0', 'amp': 2.0, 'freq': delta, 'phase': init_phase[0]+phase},
                             {'name': 'a_ch1', 'amp': 2.0, 'freq': delta, 'phase': init_phase[1]+phase}],
                             'duration' : duration}
            large_seq.append(seq_part)
        
        load_large_sin_seq(large_seq=[large_seq], identifier=[name])
        self.AWG_start()
    
    def sample_load_multi_short_sine_AWG(self, params=[]):
        """Function to create multiple single sine ensembles of set durations that is typically pi(f1), pi(f2),... that is triggered one or more 
            times at a tau instance.
            All the durations must be equal!
        """
        #Create large pulse block for the AWG - but with multiple sines
        self.AWG_reset()
        large_sequences = []
        names = []
        for cur_param in params:
            name, duration, DQ = cur_param
            delta = abs(self.LO_freq - self.target_freq)
            if DQ:
                init_phase = (100.00,0.00) # allows to drive the other transition since LO is defined to be in center in DQ methods
            else:
                init_phase = (0.00,100.00)
                
            seq_part = [{'channel_info' : [   {'name': 'a_ch0', 'amp': 2.00, 'freq': delta, 'phase': init_phase[0]},
                             {'name': 'a_ch1', 'amp': 2.00, 'freq': delta, 'phase': init_phase[1]}],
                             'duration' : duration}]
            large_sequences.append(seq_part)
            names.append(name)

        load_large_sin_seq(large_seq=large_sequences, identifier=names)
        self.AWG_start()
        
    def Rabi(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁▁▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▇▇t▇▇▁▁▁▁▁▁▁
        '''        
        name = 'rabi-juptr'
        self.tau_arr = np.linspace(tau_start, tau_stop, num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time) 
            #Pi pulse - reference
            self.ElementPS(channels={'MW_Switch':True}, length=tau)
            #Waiting time - tau
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=False)
        
        self.sample_load_large_sine_AWG(name)
    
    def T1_optical_exp(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁t▁▁▁▇▇▇▇▇
        '''
        name = 't1-opti-exp-juptr'
        self.tau_arr = np.logspace(np.log10(tau_start), np.log10(tau_stop), num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time) 
            #Waiting time - tau
            self.ElementPS(channels={'TT_Next':True}, length=tau)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=False)
    
    def T1_alt_exp_long(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁▁▁▁▁t▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
        
        Altern.
        Laser(532): ▇▇▇▇▇▁▁▁▁▁▁▁t▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁
        
        This is loading a full long sequence onto the AWG. Will be too big a sequence for large taus like 30ms.
        '''        
        name = 't1-alt-exp-long-juptr'
        self.tau_arr = np.logspace(np.log10(tau_start), np.log10(tau_stop), num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time) 
            #Pi pulse - reference
            self.ElementPS(channels={}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={'TT_Next':True}, length=tau)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi pulse - reference
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={'TT_Next':True}, length=tau)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        self.sample_load_large_sine_AWG(name)
        
    def T1_alt_exp(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁▁▁▁▁t▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
        
        Altern.
        Laser(532): ▇▇▇▇▇▁▁▁▁▁▁▁t▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁
        '''        
        name = 't1-alt-exp-juptr'
        self.tau_arr = np.logspace(np.log10(tau_start), np.log10(tau_stop), num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time) 
            #Pi pulse - reference
            self.ElementPS(channels={}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={'TT_Next':True}, length=tau)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating
            #Break after Initalisation/read out
            self.ElementPS(channels={'AWG_Trig':True}, length=self.laser_waiting_time)
            #Pi pulse - reference
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={'TT_Next':True}, length=tau)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        
        params_for_AWG_seq = []
        duration = self.pi_pulse
        #params_for_AWG_seq.append(Name_str, duration_float, DQ_bool)
        params_for_AWG_seq.append(("Pi", duration, False))
        
        self.sample_load_multi_short_sine_AWG(params=params_for_AWG_seq)
        
    def T1_dark_init_alt_exp(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁▁▁t▁▁▁▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▇pi▇▁▁▁▇pi▇▁▁▁▁▁▁▁
        
        Altern.
        Laser(532): ▇▇▇▇▇▁▁▁▁▁t▁▁▁▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁▁▁▁
        '''        
        name = 't1-dark-init-alt-exp-juptr'
        self.tau_arr = np.logspace(np.log10(tau_start), np.log10(tau_stop), num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Read out
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Waiting time - awg sync
            self.ElementPS(channels={}, length=self.awg_sync_time)
            #Pi pulse - reference
            self.ElementPS(channels={}, length=self.pi_pulse)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        
        params_for_AWG_seq = []
        duration = self.pi_pulse
        #params_for_AWG_seq.append(Name_str, duration_float, DQ_bool)
        params_for_AWG_seq.append(("Pi", duration, False))
        
        self.sample_load_multi_short_sine_AWG(params=params_for_AWG_seq)
        
    def T1_DQ_alt_exp(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁▁▁t▁▁▁▁▁▇▇▇▇▇
        MW(-1,-1):  ▁▁▁▁▁▁▁▇pi▇▁▁▁▇pi▇▁▁▁▁▁▁▁
        
        Altern.
        Laser(532): ▇▇▇▇▇▁▁▁▁▁t▁▁▁▁▁▇▇▇▇▇
        MW(-1,+1):  ▁▁▁▁▁▁▁▇pi▇▁▁▁▇pi▇▁▁▁▁▁▁▁
        '''        
        name = 't1-DQ-alt-exp-juptr'
        self.tau_arr = np.logspace(np.log10(tau_start), np.log10(tau_stop), num=tau_num)
        if self.target_freq_DQ:
            self.LO_freq = (self.target_freq_DQ-self.target_freq)/2+self.target_freq
        else:
            print('!!!Warning. DQ frequency not given!!!')
            return
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse (0,-1) - reference
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse (0,-1) - reference
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Read out
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse (0,-1) - reference
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse (0,+1) - reference!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        
        params_for_AWG_seq = []
        #params_for_AWG_seq.append(Name_str, duration_float, DQ_bool)
        params_for_AWG_seq.append(("Pi-0m1-A", self.pi_pulse, False))
        params_for_AWG_seq.append(("Pi-0m1-B", self.pi_pulse, False))
        params_for_AWG_seq.append(("Pi-0m1-A-alt", self.pi_pulse, False))
        params_for_AWG_seq.append(("Pi-0p1-B-alt", self.pi_pulse, True))
        
        self.sample_load_multi_short_sine_AWG(params=params_for_AWG_seq)
        
    def T1_SQ_alt_exp(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532): ▇▇▇▇▇▁▁▁▁▁t▁▁▁▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
        
        Altern.
        Laser(532): ▇▇▇▇▇▁▁▁▁▁t▁▁▁▁▁▇▇▇▇▇
        MW(-1):     ▁▁▁▁▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁
        '''        
        name = 't1-SQ-alt-exp-juptr'
        self.tau_arr = np.logspace(np.log10(tau_start), np.log10(tau_stop), num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time) 
            #Waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Waiting time - awg sync
            self.ElementPS(channels={}, length=self.awg_sync_time)
            #Pi pulse - reference
            self.ElementPS(channels={}, length=self.pi_pulse)
            #Read out
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Waiting time - awg sync
            self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
            #Pi pulse - reference
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)

        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        
        params_for_AWG_seq = []
        duration = self.pi_pulse
        #params_for_AWG_seq.append(Name_str, duration_float, DQ_bool)
        params_for_AWG_seq.append(("Pi", duration, False))
        
        self.sample_load_multi_short_sine_AWG(params=params_for_AWG_seq)
    
    def Ramsey_alt_phased(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532):   ▇▇▇▇▇▁▁▁▁▁▁▁▁t▁▁▁▁▁▁▁▇▇▇▇▇
        MW(-1X,-1X):  ▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁
        
        Laser(532):   ▇▇▇▇▇▁▁▁▁▁▁▁▁t▁▁▁▁▁▁▁▇▇▇▇▇
        MW(-1X,-1-X): ▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁
        '''
        name = 'ramsey-alt-phased-juptr'
        self.tau_arr = np.linspace(tau_start, tau_stop, num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #First waiting time - tau
            self.ElementPS(channels={}, length=tau)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating run
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #Second waiting time + tau
            self.ElementPS(channels={}, length=tau)
            #Pi/2 pulse Phase change cause -pi/2 pulse - done by AWG
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2, phase=180)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
        
        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        self.sample_load_large_sine_AWG(name)
    
    def Hecho_alt_phased(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁▇▇▇▇▇
        MW(-1X,-1X,-1X):  ▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁
        
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁▇▇▇▇▇
        MW(-1X,-1X,-1-X): ▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁
        '''
        name = 'hecho-alt-phased-juptr'
        self.tau_arr = np.linspace(tau_start, tau_stop, num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #First waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Second waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating run
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #First waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Second waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi/2 pulse Phase change cause -pi/2 pulse - done by AWG
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2, phase=180)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time, phase=180)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time, phase=180)
        
        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        self.sample_load_large_sine_AWG(name)
    
    def Hecho_alt(self, tau_start, tau_stop, tau_num):
        '''
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁▇▇▇▇▇
        MW(-1X,-1X,-1X):  ▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁
        
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁t/2▁▁▁▁▁▁▁▁▇▇▇▇▇
        MW(-1X,-1X,-1X): ▁▁▁▁▁▁▁▇pi/2▇▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▇3pi/2▇▁▁▁▁▁▁▁
        '''
        name = 'hecho-alt-juptr'
        self.tau_arr = np.linspace(tau_start, tau_stop, num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #First waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Second waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating run
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #First waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
            #Second waiting time + tau/2
            self.ElementPS(channels={}, length=tau/2)
            #Pi/2 pulse Phase change cause -pi/2 pulse - done by AWG
            self.ElementPS(channels={'MW_Switch':True}, length=3*self.pi_pulse/2)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
        
        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        self.sample_load_large_sine_AWG(name)
        
    def CPMG_alt_phased(self, tau_start, tau_stop, tau_num, N):
        '''
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁|▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁|▁▁▁▁▁▁▁▇▇▇▇▇
        MW:               ▁▁▁▁▁▁▁▇pi/2▇▁|▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁|▁▇pi/2▇▁▁▁▁▁▁▁
                                       X    |   t/(4*N)     Y    t/(2*N)       Y    t/(4*N)    |**N    X
        Altern.
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁|▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁|▁▁▁▁▁▁▁▇▇▇▇▇
        MW:               ▁▁▁▁▁▁▁▇pi/2▇▁|▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁|▁▇pi/2▇▁▁▁▁▁▁▁
                                       X    |   t/(4*N)     Y    t/(2*N)       Y    t/(4*N)    |**N    -X
        '''
        name = 'cpmg-alt-phased-juptr'
        self.tau_arr = np.linspace(tau_start, tau_stop, num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            
            for i in range(N):
                #First waiting time + tau/2
                self.ElementPS(channels={}, length=tau/(4*N), phase=90)
                #Pi pulse
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                #Second waiting time + tau/2
                self.ElementPS(channels={}, length=tau/(2*N), phase=90)
                #Pi pulse
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                #Second waiting time + tau/2
                self.ElementPS(channels={}, length=tau/(4*N), phase=90)
                               
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2, phase=180)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating run
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
                               
            for i in range(N):
                #First waiting time + tau/2
                self.ElementPS(channels={}, length=tau/(4*N), phase=90)
                #Pi pulse
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                #Second waiting time + tau/2
                self.ElementPS(channels={}, length=tau/(2*N), phase=90)
                #Pi pulse
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                #Second waiting time + tau/2
                self.ElementPS(channels={}, length=tau/(4*N), phase=90)
                               
            #Pi/2 pulse Phase change cause -pi/2 pulse - done by AWG
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time, phase=180)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time, phase=180)
        
        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        self.sample_load_large_sine_AWG(name)
        
    def XY8_alt_phased(self, tau_start, tau_stop, tau_num, N):
        '''
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁|▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁|▁▁▁▁▁▁▁▇▇▇▇▇
        MW:               ▁▁▁▁▁▁▁▇pi/2▇▁|▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁|▁▇pi/2▇▁▁▁▁▁▁▁
                                       X    |   t/(4*N)     Y    t/(2*N)       Y    t/(4*N)    |**N    X
        Altern.
        Laser(532):       ▇▇▇▇▇▁▁▁▁▁▁|▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁|▁▁▁▁▁▁▁▇▇▇▇▇
        MW:               ▁▁▁▁▁▁▁▇pi/2▇▁|▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁▇pi▇▁▁▁▁▁▁▁▁▁▁|▁▇pi/2▇▁▁▁▁▁▁▁
                                       X    |   t/(4*N)     Y    t/(2*N)       Y    t/(4*N)    |**N    -X
        '''
        name = 'XY8-alt-phased-juptr'
        self.tau_arr = np.linspace(tau_start, tau_stop, num=tau_num)
        
        #Create pulse sequence for the pulse streamer
        self.BlockPS = []
        self.PhaseDuration = []

        #Trigger AWG to play its sequence, which includes one complete sweep of all waiting times        
        self.ElementPS(channels={'AWG_Trig':True}, length=self.awg_sync_time)
        
        for tau in self.tau_arr:
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            
            for i in range(N):       
                self.ElementPS(channels={}, length=tau/(2*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(2*8*N))
                
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2, phase=180)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
            
            #Alternating run
            #Break after Initalisation/read out
            self.ElementPS(channels={}, length=self.laser_waiting_time)
            #Pi/2 pulse
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
                               
            for i in range(N):
                self.ElementPS(channels={}, length=tau/(2*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse)
                self.ElementPS(channels={}, length=tau/(1*8*N))
                self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse, phase=90)
                self.ElementPS(channels={}, length=tau/(2*8*N))
                               
            #Pi/2 pulse Phase change cause -pi/2 pulse - done by AWG
            self.ElementPS(channels={'MW_Switch':True}, length=self.pi_pulse/2)
            #Waiting time
            self.ElementPS(channels={'TT_Next':True}, length=self.mw_waiting_time)
            #Read out
            self.ElementPS(channels={'Laser':True, 'TT_Start':True}, length=self.read_out_time)
        
        sample_load_ready_pulsed_measurement(name=name, 
                                             element_list=self.BlockPS, tau_arr=self.tau_arr, alternating=True)
        self.sample_load_large_sine_AWG(name)
        

Helpers

In [30]:
def load_sin(channels = [{'name': 'a_ch0', 'amp': 1.00}], dur=1e-6, identifier=''):
        """
        Load a sine waveform to be played simultaneously on the specified channels.
        """
        ele = []
        a_ch = {'a_ch0': SF.DC(0), 'a_ch1': SF.DC(0), 'a_ch2': SF.DC(0), 'a_ch3': SF.DC(0)}
        d_ch = {'d_ch0': False, 'd_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False}
        for ch in channels:
            a_ch[ch['name']] = SF.Sin(amplitude=ch['amp'], frequency=ch['freq'], phase=ch['phase'])
            
        ele.append(po.PulseBlockElement(init_length_s=dur,  pulse_function=a_ch, digital_high=d_ch))
        pulse_block = po.PulseBlock(name=f'SinJupyterHecho-031123-block{identifier}', element_list=ele)
        afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().save_block(pulse_block)

        block_list = []
        block_list.append((pulse_block.name, 0))
        auto_pulse_CW = po.PulseBlockEnsemble(f'SinJupyterHecho-031123{identifier}', block_list)

        ensemble = auto_pulse_CW
        ensemblename = auto_pulse_CW.name
        afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().save_ensemble(ensemble)
        afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().sample_pulse_block_ensemble(ensemblename)
        afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().load_ensemble(ensemblename)

def load_large_sin_seq(large_seq = [[{'name': 'a_ch0', 'amp': 1.00}]], identifier=['']):
        """
        Load a large sequence to be played on the AWG.
        """
        ensemble_list = []
        for idx, ensemble in enumerate(large_seq):
            ele = []
            for step in ensemble:
                channels = step['channel_info']
                dur = step['duration']

                a_ch = {'a_ch0': SF.DC(0), 'a_ch1': SF.DC(0), 'a_ch2': SF.DC(0), 'a_ch3': SF.DC(0)}
                d_ch = {'d_ch0': False, 'd_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False}
                for ch in channels:
                    a_ch[ch['name']] = SF.Sin(amplitude=ch['amp'], frequency=ch['freq'], phase=ch['phase'])

                ele.append(po.PulseBlockElement(init_length_s=dur,  pulse_function=a_ch, digital_high=d_ch))
            pulse_block = po.PulseBlock(name=f'Jupyter-block-{identifier[idx]}', element_list=ele)
            afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().save_block(pulse_block)

            block_list = []
            block_list.append((pulse_block.name, 0))
            auto_pulse_CW = po.PulseBlockEnsemble(f'Jupyter-ensemble-{identifier[idx]}', block_list)

            ensemble = auto_pulse_CW
            ensemblename = auto_pulse_CW.name
            afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().save_ensemble(ensemble)
            afm_scanner_logic._pulsed_master_AWG.sequencegeneratorlogic().sample_pulse_block_ensemble(ensemblename)
            ensemble_list.append(ensemblename)
        print(ensemble_list)
        afm_scanner_logic._AWG.load_triggered_multi_replay(ensemble_list)
        
def sample_load_ready_pulsed_measurement(name = 'pulse_measurement', element_list = [], tau_arr = [], alternating = False):
    
    pulse_block = po.PulseBlock(name=name, element_list=element_list)
    sequencegeneratorlogic.save_block(pulse_block)
    
    block_list = []
    block_list.append((pulse_block.name, 0))
    pulse_block_ensemble = po.PulseBlockEnsemble(name, block_list)
    
    sequencegeneratorlogic.save_ensemble(pulse_block_ensemble)
    sequencegeneratorlogic.sample_pulse_block_ensemble(name)
    sequencegeneratorlogic.load_ensemble(name)
    
    tau_num = len(tau_arr) * 2 if alternating else len(tau_arr)
    pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable=tau_arr,
                                           number_of_lasers=tau_num, 
                                           laser_ignore_list=[], 
                                           alternating=alternating, 
                                           units=('s', 'arb. u.'))
    pulsedmeasurementlogic.alternative_data_type = 'None'
    time.sleep(0.5)
    pulsedmasterlogic.set_fast_counter_settings(record_length=3e-6, number_of_gates=tau_num)
    time.sleep(0.5)
    for setting, value in pulsedmasterlogic.measurement_settings.items():
        print('{0}:\n  {1}'.format(setting, value))
        
def creat_save_tag(measurement_type = 'test', tip_name = '', temperature = '', m_field = '', contact = '', extra = ''):
    save_tag = measurement_type

    if tip_name:
        save_tag += '_'+tip_name
    if temperature:
        save_tag += '_'+temperature
    if m_field:
        save_tag += '_'+m_field
    if contact:
        save_tag += '_'+contact
    if extra:
        save_tag += '_'+extra
        
    pulsedmeasurement._mw.save_tag_LineEdit.setText(save_tag)
    print('Save_tag: ',save_tag)

    
    

#### Define general measurement params

In [61]:
#measurement parameters
pi_pulse = 71e-9 #826m -- 2pi at 1.36 - pi/2 at 1.075 is 36ns -- 3pip/2 at 1.106 is 87ns

target_freq = 2.43924e9
LO_freq = target_freq + 100e6
power = -1

#Frequency for the +1 peak for DQ measurements
target_freq_DQ = None #2.97919e9
if target_freq_DQ:
    if target_freq_DQ<target_freq:
        print("!!!Ensure the DQ freq. is the higher one!!!")

#additional information for save tag
tip_name = 'A-T17-12'
temperature = '1.9K'
m_field = '200mT_Bnv'
contact = 'OOC'
extra = 'Set_B_sc99_18.98-3.22'

#### Load and ready Rabi sequence

In [60]:
tau_start = 0
tau_stop = 200e-9
tau_num = 50

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
PB.Rabi(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'Rabi', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

#### Load and ready optical T1 sequence

In [2]:
tau_start = 1e-6
tau_stop = 1e-3
tau_num = 20

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
PB.T1_optical_exp(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'T1-opt', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

#### Load and ready T1 sequence (alternating)

In [11]:
tau_start = 1e-6
tau_stop = 0.5e-3
tau_num = 10

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
PB.T1_alt_exp(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'T1-alt', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54002 GHz
Using target_freq:  2.44002 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.99473660e-06 3.97897410e-06 7.93700526e-06
 1.58322349e-05 3.15811383e-05 6.29960525e-05 1.25660531e-04
 2.50659661e-04 5.00000000e-04]
number_of_lasers:
  20
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-Pi']
Save_tag:  T1-alt_A-T17-12_1.9K_200mT_Bnv_OOC_AWGSWitch-mode_pi_pulse_calc


#### Load and ready T1 sequence init dark (alternating)

In [32]:
tau_start = 1e-6
tau_stop = 40e-3
tau_num = 20

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power, AWG_switch_mode=True)
PB.T1_dark_init_alt_exp(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'T1-dark-init-alt', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54057 GHz
Using target_freq:  2.44057 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.74668135e-06 3.05089575e-06 5.32894271e-06
 9.30796486e-06 1.62580486e-05 2.83976304e-05 4.96016115e-05
 8.66382098e-05 1.51329346e-04 2.64324146e-04 4.61690057e-04
 8.06425413e-04 1.40856823e-03 2.46031986e-03 4.29739482e-03
 7.50617940e-03 1.31109036e-02 2.29005708e-02 4.00000000e-02]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-Pi']
Save_tag:  T1-dark-init-alt_A-T17-12_1.9K_200mT_Bnv_OOC_Set_A


#### Load and ready T1 SQ sequence (alternating)

In [33]:
tau_start = 1e-6
tau_stop = 40e-3
tau_num = 20

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
PB.T1_SQ_alt_exp(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'T1-SQ-alt', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54057 GHz
Using target_freq:  2.44057 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.74668135e-06 3.05089575e-06 5.32894271e-06
 9.30796486e-06 1.62580486e-05 2.83976304e-05 4.96016115e-05
 8.66382098e-05 1.51329346e-04 2.64324146e-04 4.61690057e-04
 8.06425413e-04 1.40856823e-03 2.46031986e-03 4.29739482e-03
 7.50617940e-03 1.31109036e-02 2.29005708e-02 4.00000000e-02]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-Pi']
Save_tag:  T1-SQ-alt_A-T17-12_1.9K_200mT_Bnv_OOC_Set_A


#### Load and ready T1 DQ sequence (alternating)

In [13]:
tau_start = 1e-6
tau_stop = 1e-3
tau_num = 20

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, target_freq_DQ=target_freq_DQ, power=power)
PB.T1_DQ_alt_exp(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'T1-DQ-alt', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

Using laser voltage:  0.07 V
Using LO_freq:  2.881845 GHz
Using target_freq:  2.78469 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.43844989e-06 2.06913808e-06 2.97635144e-06
 4.28133240e-06 6.15848211e-06 8.85866790e-06 1.27427499e-05
 1.83298071e-05 2.63665090e-05 3.79269019e-05 5.45559478e-05
 7.84759970e-05 1.12883789e-04 1.62377674e-04 2.33572147e-04
 3.35981829e-04 4.83293024e-04 6.95192796e-04 1.00000000e-03]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
Save_tag:  T1-DQ-alt_A-T17-12_1.9K_0mT_Bnv_OOC


#### Load and ready Ramsey sequence with phase shift (alternating)

In [49]:
tau_start = 0
tau_stop = 2e-6
tau_num = 50

PB = qudi_pulse_block(pi_pulse=pi_pulse, pi_half_pulse=pi_half_pulse, three_pi_half_pulse=three_pi_half_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
PB.Ramsey_alt_phased(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'Ramsey-alt-phased', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54002 GHz
Using target_freq:  2.44002 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [0.00000000e+00 4.08163265e-08 8.16326531e-08 1.22448980e-07
 1.63265306e-07 2.04081633e-07 2.44897959e-07 2.85714286e-07
 3.26530612e-07 3.67346939e-07 4.08163265e-07 4.48979592e-07
 4.89795918e-07 5.30612245e-07 5.71428571e-07 6.12244898e-07
 6.53061224e-07 6.93877551e-07 7.34693878e-07 7.75510204e-07
 8.16326531e-07 8.57142857e-07 8.97959184e-07 9.38775510e-07
 9.79591837e-07 1.02040816e-06 1.06122449e-06 1.10204082e-06
 1.14285714e-06 1.18367347e-06 1.22448980e-06 1.26530612e-06
 1.30612245e-06 1.34693878e-06 1.38775510e-06 1.42857143e-06
 1.46938776e-06 1.51020408e-06 1.55102041e-06 1.59183673e-06
 1.63265306e-06 1.67346939e-06 1.71428571e-06 1.75510204e-06
 1.79591837e-06 1.83673469e-06 1.87755102e-06 1.91836735e-06
 1.95918367e-06 2.00000000e-06]
number_of_lasers:
  100
laser_ignore_list:
  []
alternating:
  True
units:
  (

#### Load and ready Hahn Echo sequence with phase shift (alternating)

In [62]:
tau_start = 200e-9
tau_stop = 10e-6
tau_num = 100

PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
PB.Hecho_alt_phased(tau_start, tau_stop, tau_num)
creat_save_tag(measurement_type = 'Hecho-alt-phased', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

#### Load and ready CPMG sequence (alternating)

In [39]:
4*64*pi_pulse

1.8944e-05

In [63]:
tau_start = 600e-9
tau_stop = 20e-6
tau_num = 200
N = 2

if tau_start/(4*N) < pi_pulse:
    min_tau = pi_pulse*2*2*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
    PB.CPMG_alt_phased(tau_start, tau_stop, tau_num, N)
    creat_save_tag(measurement_type = f'CPMG({N})-alt-phased', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

#### Load and ready XY8 sequence (alternating)

In [49]:
tau_start = 1.92e-5
tau_stop = 50e-6
tau_num = 200
N = 16

if tau_start/(2*8*N) < pi_pulse:
    min_tau = pi_pulse*2*8*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    PB = qudi_pulse_block(pi_pulse=pi_pulse, LO_freq=LO_freq, target_freq=target_freq, power=power)
    PB.XY8_alt_phased(tau_start, tau_stop, tau_num, N)
    creat_save_tag(measurement_type = f'XY8({N})-alt-phased', tip_name = tip_name, temperature = temperature, m_field = m_field, contact = contact, extra = extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54057 GHz
Using target_freq:  2.44057 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.92000000e-05 1.93547739e-05 1.95095477e-05 1.96643216e-05
 1.98190955e-05 1.99738693e-05 2.01286432e-05 2.02834171e-05
 2.04381910e-05 2.05929648e-05 2.07477387e-05 2.09025126e-05
 2.10572864e-05 2.12120603e-05 2.13668342e-05 2.15216080e-05
 2.16763819e-05 2.18311558e-05 2.19859296e-05 2.21407035e-05
 2.22954774e-05 2.24502513e-05 2.26050251e-05 2.27597990e-05
 2.29145729e-05 2.30693467e-05 2.32241206e-05 2.33788945e-05
 2.35336683e-05 2.36884422e-05 2.38432161e-05 2.39979899e-05
 2.41527638e-05 2.43075377e-05 2.44623116e-05 2.46170854e-05
 2.47718593e-05 2.49266332e-05 2.50814070e-05 2.52361809e-05
 2.53909548e-05 2.55457286e-05 2.57005025e-05 2.58552764e-05
 2.60100503e-05 2.61648241e-05 2.63195980e-05 2.64743719e-05
 2.66291457e-05 2.67839196e-05 2.69386935e-05 2.70934673e-05
 2.72482412e-05 2.74030151e-05 2.75577889e-05 2.7

In [43]:
16*1*pi_pulse

1.184e-06